<a href="https://colab.research.google.com/github/Open-Sourced-Olaf/DocVerifier/blob/main/privacy_policy_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

In [ ]:
good_privacy = []
with open("good_privacy.txt", "r") as f:
  for line in f:
    good_privacy.append(line.rstrip())
bad_privacy_raw = []
with open("bad_privacy.txt", "r") as f:
  for line in f:
    bad_privacy_raw.append(line.rstrip())

In [ ]:
bad_privacy = bad_privacy_raw[: -6 or None]

In [ ]:
from sklearn.model_selection import train_test_split
# splitting the model into X and Y training and test data
# X is good, Y is bad
# we will represent X with 0 and Y with 1
X_train, X_test, y_train, y_test = train_test_split(good_privacy, bad_privacy, test_size=0.2, random_state=42)

In [ ]:
combined_train = []
for item in X_train:
  combined_train.append([item, 0])
for item in y_train:
  combined_train.append([item, 1])
random.shuffle(combined_train)

training_data = []
training_target = []
for item in combined_train:
  training_data.append(item[0])
  training_target.append(item[1])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])
text_clf = text_clf.fit(training_data, training_target)

In [ ]:
combined_test = []
for item in X_test:
  combined_test.append([item, 0])
for item in y_test:
  combined_test.append([item, 1])
random.shuffle(combined_test)

test_data = []
test_target = []
for item in combined_test:
  test_data.append(item[0])
  test_target.append(item[1])

In [ ]:
import numpy as np
predicted = text_clf.predict(test_data)
np.mean(predicted == test_target)

0.6111111111111112

In [ ]:
for idx, item in enumerate(test_data):
  print(test_data[idx], 'predicted:', predicted[idx], 'actual:', test_target[idx])

You must provide your legal name, pseudonyms are not allowed. predicted: 1 actual: 1
Mozilla does not receive or store your payment details. predicted: 0 actual: 0
Choosing whether we share your personal information with other companies so they can send you offers and promotions about their products and services predicted: 1 actual: 0
We collect purchase history. predicted: 1 actual: 1
We use Pixels, web beacons, clear GIFs etc in our emails to let us know when they have been opened or forwarded, so we can gauge the effectiveness of our communications. predicted: 0 actual: 1
We will clearly and transparently disclose our data policies and practices to our users predicted: 1 actual: 0
You can edit or delete your account at any time. predicted: 0 actual: 0
We may use cookies to show you relevant advertising both on and off the LinkedIn site. We may also use a cookie to learn whether someone who saw an ad later visited and took an action (e.g. downloaded a white paper or made a purchase) 

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                        alpha=1e-3, random_state=42))
])
text_clf_svm = text_clf_svm.fit(training_data, training_target)
predicted_svm = text_clf_svm.predict(test_data)
np.mean(predicted_svm == test_target)

0.6666666666666666

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
 }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(training_data, training_target)
gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [ ]:
predicted_gs_clf = gs_clf.predict(test_data)
np.mean(predicted_gs_clf == test_target)

0.7222222222222222

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf-svm__alpha': (1e-2, 1e-3),
 }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(training_data, training_target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [ ]:
predicted_gs_clf_svm = gs_clf_svm.predict(test_data)
np.mean(predicted_gs_clf_svm == test_target)

0.6666666666666666